Extract title, price, and description--ultiamtely from a second link from first page only


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Get the html from the page
url = 'https://books.toscrape.com/'
response = requests.get(url)
print(response.status_code)
response.ok

200


True

In [3]:
# Make a beautiful soup
soup = BeautifulSoup(response.content)
print(soup.prettify()[:500])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta conte


In [4]:
# Get the list of books (use the inspector tool on the website to help locate)
books = soup.find_all('li', {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"}) 
books

[<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
 <article class="product_pod">
 <div class="image_container">
 <a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
 </div>
 <p class="star-rating Three">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>
 <h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
 <div class="product_price">
 <p class="price_color">£51.77</p>
 <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>
 <form>
 <button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
 </form>
 </div>
 </article>
 </li>,
 <li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
 <article class="product_pod">
 <div class="image_

In [5]:
len(books)

20

In [6]:
# Explore first book
book = books[0]
book

<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>
</li>

In [7]:
# Extract the title
# It is in an h3 tag
h3= book.find('h3')
h3

<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>

In [8]:
a_tag = h3.find('a')
a_tag

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [9]:
title = a_tag['title']
title

'A Light in the Attic'

In [10]:
# Chain together
title = book.find('h3').find('a')['title']
title

'A Light in the Attic'

In [11]:
# # Can also use
# title = book.h3.a['title']
# title

In [12]:
# Extract price
# It is i p tag with price_color
price = book.find('p', {'class': 'price_color'})
price

<p class="price_color">£51.77</p>

In [13]:
# Clean it up to just the text
price.text.strip()

'£51.77'

In [14]:
# Extract link to description
links = book.find_all('a', href = True)
links

[<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>,
 <a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>]

In [15]:
# The two links have the same adderess so either will work
link = links[0].attrs['href']
link

'catalogue/a-light-in-the-attic_1000/index.html'

In [16]:
# We can already see that this is not an entire link! 
# It is a relative link that requires the "books.toscrape.com/" before the link will work. We can add this later

In [17]:
# now we need to put this in a loop to find the data for all of the books

books_data = []

for book in books:
    # Extract the title
    title = book.find('h3').find('a')['title']

    # Extract price
    price = book.find('p', {'class': 'price_color'}).text.strip()

    # Extract link to description
    links = book.find_all('a', href = True)
    link = links[0].attrs['href']

    # Organize into a dictionary
    book_dict = {'Title': title,
                'Price' : price,
                'Details_link' : link}
    books_data.append(book_dict)


In [18]:
# Create the dataframe
df = pd.DataFrame(books_data)
df.head()

,Title,Price,Details_link
0,A Light in the Attic,£51.77,catalogue/a-light-in-the-attic_1000/index.html
1,Tipping the Velvet,£53.74,catalogue/tipping-the-velvet_999/index.html
2,Soumission,£50.10,catalogue/soumission_998/index.html
3,Sharp Objects,£47.82,catalogue/sharp-objects_997/index.html
4,Sapiens: A Brief History of Humankind,£54.23,catalogue/sapiens-a-brief-history-of-humankind...


In [19]:
# We need to add books.toscrape.com/ to each Details_link
# Sart by defining the base url
base_url = 'https://books.toscrape.com/'

# now add the base url to each relative url
df['Complete_link'] = base_url + df['Details_link']
df

,Title,Price,Details_link,Complete_link
0,A Light in the Attic,£51.77,catalogue/a-light-in-the-attic_1000/index.html,https://books.toscrape.com/catalogue/a-light-i...
1,Tipping the Velvet,£53.74,catalogue/tipping-the-velvet_999/index.html,https://books.toscrape.com/catalogue/tipping-t...
2,Soumission,£50.10,catalogue/soumission_998/index.html,https://books.toscrape.com/catalogue/soumissio...
3,Sharp Objects,£47.82,catalogue/sharp-objects_997/index.html,https://books.toscrape.com/catalogue/sharp-obj...
4,Sapiens: A Brief History of Humankind,£54.23,catalogue/sapiens-a-brief-history-of-humankind...,https://books.toscrape.com/catalogue/sapiens-a...
5,The Requiem Red,£22.65,catalogue/the-requiem-red_995/index.html,https://books.toscrape.com/catalogue/the-requi...
6,The Dirty Little Secrets of Getting Your Dream...,£33.34,catalogue/the-dirty-little-secrets-of-getting-...,https://books.toscrape.com/catalogue/the-dirty...
7,The Coming Woman: A Novel Based on the Life of...,£17.93,catalogue/the-coming-woman-a-novel-based-on-th...,https://books.toscrape.com/catalogue/the-comin...
8,The Boys in the Boat: Nine Americans and Their...,£22.60,catalogue/the-boys-in-the-boat-nine-americans-...,https://books.toscrape.com/catalogue/the-boys-...
9,The Black Maria,£52.15,catalogue/the-black-maria_991/index.html,https://books.toscrape.com/catalogue/the-black...


In [23]:
# Open link to extract description
sites = df['Complete_link']
site = sites[0]

In [24]:
# Get the job page's response
import time

response = requests.get(site)
time.sleep(2)

In [25]:
response.ok

True

In [27]:
soup = BeautifulSoup(response.content)
print(soup.prettify()[:500])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   A Light in the Attic | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <me


In [31]:
from IPython.core.display import HTML
# Visualizing the page
HTML(str(soup))

UPC,a897fe39b1053632
Product Type,Books
Price (excl. tax),£51.77
Price (incl. tax),£51.77
Tax,£0.00
Availability,In stock (22 available)
Number of reviews,0
